In [27]:
%pip install requests

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import re
import pandas as pd
import requests


In [ ]:
# ./export/hospital/2021_2024_09_trainDatas.csv -> 병원 모델
# ./export/education/final_graded_school_locations.csv -> 교육 모델

In [3]:


def get_coordinates_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            location = result["documents"][0]["address"]
            if location is None:
                return None, None
            return float(location["y"]), float(location["x"])  # 위도, 경도 반환
        else:
            print(f"해당 주소에 대한 좌표 정보를 찾을 수 없습니다. {address}")
            return None, None
    else:
        print( f"에러 발생: {response.status_code}")
        return None, None


In [4]:
#병원 점수 예측 데이터
trainedHospital = pd.read_csv('../export/hospital/2021_2024_09_trainDatas.csv')


# 주소 컬럼에서 괄호 안의 내용과 괄호를 제거
#trainedHospital['주소'] = trainedHospital['주소'].str.replace(r'\(.*?\)', '', regex=True)

# 결과 출
trainedHospital.head()

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,주소,...,좌표(X),좌표(Y),equitments,levels,grade,medicalLevel,cluster,DistanceToCenter,score,class
0,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,경희대학교병원,1,상급종합,110000,서울,110007,동대문구,NaN,서울특별시 동대문구 경희대로 23 (회기동),...,NaN,NaN,0.208719,0.071429,1.0,0.910256,1,0.146532,1.382311,0.0
1,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,한양대학교병원,1,상급종합,110000,서울,110011,성동구,NaN,서울특별시 성동구 왕십리로 222-1 (사근동),...,NaN,NaN,0.164827,0.071429,1.0,0.884615,1,0.141688,1.377467,0.0
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,학교법인 고려중앙학원 고려대학교의과대학부속병원(안암병원),1,상급종합,110000,서울,110012,성북구,NaN,서울특별시 성북구 고려대로 73 고려대병원 (안암동5가),...,NaN,NaN,0.218374,0.071429,1.0,0.948718,1,0.168272,1.404051,0.0
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,NaN,서울특별시 종로구 새문안로 29 (평동),...,NaN,NaN,0.190506,0.071429,1.0,0.923077,1,0.153161,1.388940,0.0
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQyIy...,학교법인가톨릭학원가톨릭대학교서울성모병원,1,상급종합,110000,서울,110021,서초구,NaN,서울특별시 서초구 반포대로 222 (반포동),...,NaN,NaN,0.346420,0.071429,1.0,0.891026,1,0.199852,1.435631,0.0


In [ ]:




block = 0
le = 0
# 사용 예제


for i in range(len(trainedHospital)):
    address = trainedHospital.iloc[i]['주소']
    address = address.split(' ')
    address = ' '.join(address[:4])
    coord = get_coordinates_kakao(address,'api key')
    trainedHospital.at[i,'lat'] = coord[0]
    trainedHospital.at[i,'lon'] = coord[1]

    block += 1
    if block > 100:
        print("--- " ,block , "  : ",le)
        block = 0
        le += 1

해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99
---  101   :  0
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 강남구 논현로 507-507
---  101   :  1
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 도봉구 삼양로 606-606
---  101   :  2
---  101   :  3
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99
---  101   :  4
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306
---  101   :  5
---  101   :  6
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99,
---  101   :  7
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306,
---  101   :  8
---  101   :  9
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99,
---  101   :  10
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306,
---  101   :  11
---  101   :  12
---  101   :  13
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306,
---  101   :  14
---  101   :  15
---  101   :  16
---  101   :  17


In [6]:
trainedHospital

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,주소,...,equitments,levels,grade,medicalLevel,cluster,DistanceToCenter,score,class,lat,lon
0,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,경희대학교병원,1,상급종합,110000,서울,110007,동대문구,NaN,서울특별시 동대문구 경희대로 23 (회기동),...,0.208719,0.071429,1.0,0.910256,1,0.146532,1.382311,0.0,37.593776,127.050742
1,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,한양대학교병원,1,상급종합,110000,서울,110011,성동구,NaN,서울특별시 성동구 왕십리로 222-1 (사근동),...,0.164827,0.071429,1.0,0.884615,1,0.141688,1.377467,0.0,37.558582,127.045716
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,학교법인 고려중앙학원 고려대학교의과대학부속병원(안암병원),1,상급종합,110000,서울,110012,성북구,NaN,서울특별시 성북구 고려대로 73 고려대병원 (안암동5가),...,0.218374,0.071429,1.0,0.948718,1,0.168272,1.404051,0.0,37.587377,127.026249
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,NaN,서울특별시 종로구 새문안로 29 (평동),...,0.190506,0.071429,1.0,0.923077,1,0.153161,1.388940,0.0,37.568868,126.967813
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQyIy...,학교법인가톨릭학원가톨릭대학교서울성모병원,1,상급종합,110000,서울,110021,서초구,NaN,서울특별시 서초구 반포대로 222 (반포동),...,0.346420,0.071429,1.0,0.891026,1,0.199852,1.435631,0.0,37.502394,127.005861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860,JDQ4MTg4MSM1MSMkMiMkNCMkMDAkNTgxOTYxIzUxIyQxIy...,이담외과의원,31,의원,110000,서울,110001,강남구,신사동,"서울특별시 강남구 도산대로 235, 평화빌딩 2,3층 (신사동)",...,0.014518,0.285714,0.4,0.089744,0,0.186853,0.841253,2.0,37.521588,127.032643
1861,JDQ4MTg4MSM1MSMkMiMkNCMkMDAkNTgxOTYxIzQxIyQxIy...,제이본정형외과의원,31,의원,110000,서울,110020,양천구,목동,"서울특별시 양천구 목동중앙북로 68, 2층 201~216호 (목동, 목동에버하임)",...,0.027052,0.428571,0.4,0.025641,0,0.256795,0.911195,2.0,37.546575,126.870684
1862,JDQ4MTg4MSM1MSMkMiMkNCMkMDAkNDgxNzAyIzIxIyQxIy...,최상산부인과의원,31,의원,110000,서울,110021,서초구,서초동,"서울특별시 서초구 서초대로 351, B1,4~8층 (서초동)",...,0.018357,0.142857,0.4,0.115385,0,0.225670,0.880070,2.0,37.495795,127.019673
1863,JDQ4MTg4MSM1MSMkMiMkOCMkMDAkMzgxOTYxIzIxIyQxIy...,태릉마이크로의원,31,의원,110000,서울,110019,중랑구,묵동,"서울특별시 중랑구 동일로 872, 지하1~6층 (묵동)",...,0.023354,0.428571,0.4,0.076923,0,0.245123,0.899524,2.0,37.608417,127.078466


In [7]:
#trainedHospital.to_csv('../export/safety/policeOutput.csv', index=False, encoding='utf-8') 
trainedHospital.to_csv('../export/hospital/hospitalOutput.csv', index=False, encoding='utf-8') 